In [68]:
import uproot
import awkward as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from tqdm.notebook import tqdm
from collections import Counter

SEQ_LENGTH = 3
COLLIDING_BUNCHES = False

In [69]:
if COLLIDING_BUNCHES:
    filepath = f"data-sequences/output_1000_seq{SEQ_LENGTH}.root"

else:
    filepath = f"data-sequences/output_1000_seq{SEQ_LENGTH}_nc.root"

data = uproot.open(filepath + ":L1BMTFStubSequences").arrays()
df = ak.to_dataframe(data)
df.head()

nL1BMTFStub  L1BMTFStub_hwQual  L1BMTFStub_hwPhi  \
entry subentry                                                     
0     0                   1                  3               622   
1     0                   1                  3               623   
2     0                   1                  2               -22   
3     0                   2                  6              1104   
      1                   2                  3              -176   

                L1BMTFStub_hwPhiB  L1BMTFStub_hwEta  L1BMTFStub_hwQEta  \
entry subentry                                                           
0     0                      -300                 0                  0   
1     0                      -300                 0                  0   
2     0                       -76                 0                  0   
3     0                      -152                 0                  0   
      1                      -201                 0                  0   

                L1BMTFStub_wheel  L1BMTFStub_sector  L1BMTFStub_station  \
entry subentry                                                            
0     0                       -2                  6                   2   
1     0                       -2                  6                   2   
2     0                        2                  6                   4   
3     0                        1                  3                   4   
      1                        1                  4                   1   

                orbitNumber  bunchCrossing  sequenceIndex  
entry subentry                                             
0     0           261882734           2463              0  
1     0           261882734           2464              0  
2     0           261882734           2465              0  
3     0           261884234           2112              1  
      1           261884234           2112              1

In [70]:
dfgb = df.groupby("sequenceIndex")

index = 12

print(f"Index: {index}")
dfg = dfgb.get_group(index)
dfg_selected = dfg[["bunchCrossing", "L1BMTFStub_wheel", "L1BMTFStub_station"]]
print(dfg_selected)

print(f"\n\nSequence: {index}")
for sector, dfsector in dfg.groupby("L1BMTFStub_sector"):
    print(f'\tSector: {sector}, Wheel: {dfsector["L1BMTFStub_wheel"].values}, Station: {dfsector["L1BMTFStub_station"].values}, BX: {dfsector["bunchCrossing"].values}')

Index: 12
                bunchCrossing  L1BMTFStub_wheel  L1BMTFStub_station
entry subentry                                                     
36    0                  3138                 1                   3
      1                  3138                 1                   2
      2                  3138                 1                   4
      3                  3138                 2                   3
37    0                  3139                 1                   3
      1                  3139                 1                   1
38    0                  3140                 2                   4


Sequence: 12
	Sector: 2, Wheel: [1 1 1], Station: [3 2 3], BX: [3138 3138 3139]
	Sector: 3, Wheel: [1], Station: [4], BX: [3138]
	Sector: 11, Wheel: [2 1 2], Station: [3 1 4], BX: [3138 3139 3140]


In [71]:
[
    tuple(el.item() for el in key) 
    for key in dfg_selected.groupby(["bunchCrossing", "L1BMTFStub_station"]).indices
    ]

[(3138, 2), (3138, 3), (3138, 4), (3139, 1), (3139, 3), (3140, 4)]

In [72]:
def getSameBxTargetCollections(bxs):
    target_pairs = [
        [(bx, station) for station in range(ii, ii + SEQ_LENGTH)]
        for bx in bxs
        for ii in range(1, 4 - SEQ_LENGTH + 2)
    ]
    
    return target_pairs


# interesting ones !!!
def getMultiBxTargetCollections(bxs):
    target_pairs = [
        [(bx, station) for bx, station in zip(bxs, range(ii, ii + SEQ_LENGTH))]
        for ii in range(1, 4 - SEQ_LENGTH + 2)
    ]

    return target_pairs

def getInverseMultiBxTargetCollections(bxs):
    target_pairs = [
        [(bx, station) for bx, station in zip(bxs, range(ii + SEQ_LENGTH, ii, -1))]
        for ii in range(0, 4 - SEQ_LENGTH + 1)
    ]

    return target_pairs

In [73]:
multi_bx_selection = {}
inverse_multi_bx_selection = {}

for idx_seq, df_seq in tqdm(df.groupby("sequenceIndex")):
    pairs = [
        tuple(el.item() for el in key) 
        for key in df_seq.groupby(["bunchCrossing", "L1BMTFStub_station"]).indices
    ]

    multi_bx_collections = getMultiBxTargetCollections(df_seq["bunchCrossing"].unique().tolist())

    for tps in multi_bx_collections:
        if np.all([target_pair in pairs for target_pair in tps]):
            """
            Here we keep only the entries of the sequence sub-dataframe that 
            correspond to the identified (bx, station) pairs.
            This allows us then to groupby wheel and keep only the wheels that are
            at most distant 1 to one another.
            """
            mi = pd.MultiIndex.from_tuples(tps, names=["bunchCrossing", "L1BMTFStub_station"])
            df_temp = df_seq[
                df_seq.set_index(["bunchCrossing", "L1BMTFStub_station"]).index.isin(mi)
            ]

            if df_temp["L1BMTFStub_wheel"].max() - df_temp["L1BMTFStub_wheel"].min() > 1:
                if df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                    if df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                        continue

                    else:
                        df_temp = df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]
                        multi_bx_selection[idx_seq] = df_temp

                else:
                    df_temp = df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]
                    multi_bx_selection[idx_seq] = df_temp

            else:
                multi_bx_selection[idx_seq] = df_temp

    inverse_multi_bx_collections = getInverseMultiBxTargetCollections(df_seq["bunchCrossing"].unique().tolist())

    for tps in inverse_multi_bx_collections:
        if np.all([target_pair in pairs for target_pair in tps]):
            """
            Here we keep only the entries of the sequence sub-dataframe that 
            correspond to the identified (bx, station) pairs.
            This allows us then to groupby wheel and keep only the wheels that are
            at most distant 1 to one another.
            """
            mi = pd.MultiIndex.from_tuples(tps, names=["bunchCrossing", "L1BMTFStub_station"])
            df_temp = df_seq[
                df_seq.set_index(["bunchCrossing", "L1BMTFStub_station"]).index.isin(mi)
            ]

            if df_temp["L1BMTFStub_wheel"].max() - df_temp["L1BMTFStub_wheel"].min() > 1:
                if df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                    if df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]["bunchCrossing"].nunique() < SEQ_LENGTH:
                        continue

                    else:
                        df_temp = df_temp[df_temp["L1BMTFStub_wheel"] > df_temp["L1BMTFStub_wheel"].min()]
                        inverse_multi_bx_selection[idx_seq] = df_temp

                else:
                    df_temp = df_temp[df_temp["L1BMTFStub_wheel"] < df_temp["L1BMTFStub_wheel"].max()]
                    inverse_multi_bx_selection[idx_seq] = df_temp

            else:
                inverse_multi_bx_selection[idx_seq] = df_temp

  0%|          | 0/444 [00:00<?, ?it/s]

In [74]:
df_tot_mbs = pd.concat(list(multi_bx_selection.values()))
df_tot_imbs = pd.concat(list(inverse_multi_bx_selection.values()))

In [88]:
if COLLIDING_BUNCHES:
    pkl_mbs = f"df_mbs_output_1000_seq{SEQ_LENGTH}.pkl"
    pkl_imbs = f"df_imbs_output_1000_seq{SEQ_LENGTH}.pkl"

else:
    pkl_mbs = f"df_mbs_output_1000_seq{SEQ_LENGTH}_nc.pkl"
    pkl_imbs = f"df_imbs_output_1000_seq{SEQ_LENGTH}_nc.pkl"

df_tot_mbs.to_pickle("selected-sequences/" + pkl_mbs)
df_tot_imbs.to_pickle("selected-sequences/" + pkl_imbs)